In [2]:
import json
import os

In [3]:
path = "ExperimentData//"
path = "SampleSessionLayerableOnly//"
dir_list = os.listdir(path)
 
print("Files and directories in '", path, "' :")
 
# prints all files
print(dir_list)


Files and directories in ' SampleSessionLayerableOnly// ' :
['RecordStudy_Session_0_1652384297714.json', 'RecordStudy_Session_10_1652384297714.json', 'RecordStudy_Session_11_1652384297714.json', 'RecordStudy_Session_12_1652384297714.json', 'RecordStudy_Session_13_1652384297714.json', 'RecordStudy_Session_14_1652384297714.json', 'RecordStudy_Session_15_1652384297714.json', 'RecordStudy_Session_16_1652384297714.json', 'RecordStudy_Session_17_1652384297714.json', 'RecordStudy_Session_18_1652384297714.json', 'RecordStudy_Session_19_1652384297714.json', 'RecordStudy_Session_1_1652384297714.json', 'RecordStudy_Session_20_1652384297714.json', 'RecordStudy_Session_21_1652384297714.json', 'RecordStudy_Session_22_1652384297714.json', 'RecordStudy_Session_23_1652384297714.json', 'RecordStudy_Session_24_1652384297714.json', 'RecordStudy_Session_25_1652384297714.json', 'RecordStudy_Session_26_1652384297714.json', 'RecordStudy_Session_27_1652384297714.json', 'RecordStudy_Session_28_1652384297714.jso

In [4]:
appEvents = []
codesEvents = []
experimentEvents = []

def parseData(json_data):
    frames = json_data["frames"]
    for frame in frames:
        appEvent = frame["appEvents"]
        codeEvent = frame["codesEvents"]
        experimentEvent = frame["experimentEvents"]
        if appEvent:
            appEvents.append(appEvent)
        if codeEvent:
            codesEvents.append(codeEvent)
        if experimentEvent:
            experimentEvents.append(experimentEvent)


In [5]:

for filename in dir_list:
    data = []
    if filename.endswith(".json"):
        # Prints only text file present in My Folder
        print(filename)
        firstFileFlag = False
        if "Session_0" in filename:
            firstFileFlag = True
        with open(path + filename, 'r') as f:
            json_data = json.load(f)
            if(firstFileFlag):
                #data.append(json_data["obj"]["sessionRecording"])
                parseData(json_data["obj"]["sessionRecording"])
            else:
                #data.append(json.load(f))
                parseData(json_data)

RecordStudy_Session_0_1652384297714.json
RecordStudy_Session_10_1652384297714.json
RecordStudy_Session_11_1652384297714.json
RecordStudy_Session_12_1652384297714.json
RecordStudy_Session_13_1652384297714.json
RecordStudy_Session_14_1652384297714.json
RecordStudy_Session_15_1652384297714.json
RecordStudy_Session_16_1652384297714.json
RecordStudy_Session_17_1652384297714.json
RecordStudy_Session_18_1652384297714.json
RecordStudy_Session_19_1652384297714.json
RecordStudy_Session_1_1652384297714.json
RecordStudy_Session_20_1652384297714.json
RecordStudy_Session_21_1652384297714.json
RecordStudy_Session_22_1652384297714.json
RecordStudy_Session_23_1652384297714.json
RecordStudy_Session_24_1652384297714.json
RecordStudy_Session_25_1652384297714.json
RecordStudy_Session_26_1652384297714.json
RecordStudy_Session_27_1652384297714.json
RecordStudy_Session_28_1652384297714.json
RecordStudy_Session_29_1652384297714.json
RecordStudy_Session_2_1652384297714.json
RecordStudy_Session_30_1652384297714.

In [6]:

#experimentEvents[0][0]
experimentEvents
pinCodes = experimentEvents[0][0]["pinCodes"]

for exper in experimentEvents:
    for exp in exper:
        if exp["eventName"] == "EndTask":
            print(exp)


{'unixTime': 1652385447237, 'systemTime': '12-57-27-23', 'userID': '1652384297714', 'sessionType': 'layerable', 'eventName': 'EndTask', 'pinRandomSeed': 7088, 'pinNumCodes': 200, 'pinCodeLength': 6, 'pinNumPieces': 3, 'pinCodes': [], 'score': 14}


In [25]:
import pandas
pandas.options.display.max_rows = None

In [46]:
experimentEvents
experimentsUnpacked = [event[0] for event in experimentEvents]

experimentsDf = pandas.DataFrame(experimentsUnpacked)
experimentsDf

,unixTime,systemTime,userID,sessionType,eventName,pinRandomSeed,pinNumCodes,pinCodeLength,pinNumPieces,pinCodes,score
0,1652384847234,12-47-27-23,1652384297714,layerable,StartTask,7088,200,6,3,"[{'Value': 709182, 'Label': 'IU', 'Pieces': [{...",0
1,1652385447237,12-57-27-23,1652384297714,layerable,EndTask,7088,200,6,3,[],14
2,1652385466769,12-57-46-76,1652384297714,immersive,StartTask,7088,200,6,3,"[{'Value': 709182, 'Label': 'IU', 'Pieces': [{...",0


In [86]:
appEvents
appsUnpacked = [appEvent[0] for appEvent in appEvents]

appsDf = pandas.DataFrame(appsUnpacked)
appsDf.sort_values(by='eventTime')

tutorialEvents = appsDf[appsDf['name'] == 'Tutorial']
annotationEvents = appsDf[appsDf['name'] == 'AnnotationsMenu']
codeEntryEvents = appsDf[appsDf['name'] == 'Code Entry']
itemInspectorEvents = appsDf[appsDf['name'] == 'Item Inspector']
deviceGroupsEvents = appsDf[appsDf['name'] == 'Device Groups']
atlasEvents = appsDf[appsDf['name'] == 'Atlas']

atlasStarts = atlasEvents[atlasEvents['activity'] == 'Start']
atlasStops = atlasEvents[atlasEvents['activity'] == 'Stop']

layerableStartTime = experimentsDf.loc[(experimentsDf['sessionType'] == 'layerable') & (experimentsDf['eventName'] == 'StartTask')]['unixTime'].to_numpy()[0]
layerableEndTime = experimentsDf.loc[(experimentsDf['sessionType'] == 'layerable') & (experimentsDf['eventName'] == 'EndTask')]['unixTime'].to_numpy()[0]

layerableCodeEntryEvents = codeEntryEvents.loc[(codeEntryEvents['unixTime'] > layerableStartTime) & (codeEntryEvents['unixTime'] < layerableEndTime)]
layerableCodeEntryEvents
atlasEvents

,unixTime,systemTime,eventTime,activityID,activityType,activity,name
1,1652384683912,12-44-43-91,386198,f5aa6c54-feef-4c63-9662-3a8ccda814ca,1,Start,Atlas
6,1652384837684,12-47-17-68,539970,f5aa6c54-feef-4c63-9662-3a8ccda814ca,1,Stop,Atlas
10,1652384865385,12-47-45-38,567671,f5aa6c54-feef-4c63-9662-3a8ccda814ca,1,Start,Atlas
33,1652385462278,12-57-42-27,1164564,f5aa6c54-feef-4c63-9662-3a8ccda814ca,1,Stop,Atlas
36,1652385484940,12-58-04-93,1187226,f5aa6c54-feef-4c63-9662-3a8ccda814ca,1,Start,Atlas
45,1652385546033,12-59-06-03,1248319,f5aa6c54-feef-4c63-9662-3a8ccda814ca,1,Start,Atlas
50,1652385581305,12-59-41-30,1283591,f5aa6c54-feef-4c63-9662-3a8ccda814ca,1,Start,Atlas
